In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller
import shutil
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoSuchElementException
from time import sleep

import re
import os
import pandas as pd

In [ ]:
# 처음 1번만 설치.
pip install chromedriver-autoinstaller

In [ ]:
def createDirectory(directory): 
    """ 새로운 폴더를 생성하는 함수 """
    try: 
        if not os.path.exists(directory): 
            os.makedirs(directory) 
    except OSError: 
        print("Error: Failed to create the directory.")

In [ ]:
def chromedriver_update():
    """ 현재 디렉토리에 자동으로 최신 버전 chromedriver를 생성하는 함수.
    최신 버전 number를 폴더 이름으로 하여 그 폴더 안에 생성됨.
    현재 최신 버전 number가 아닌 이전 버전은 자동으로 삭제하도록 설계. """
    
    # 최신 버전 number. 2022-01-14 기준 chrome_ver : 97
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    current_list = os.listdir(os.getcwd()) 
    chrome_list = []
    
    # 현재 디렉토리 안에 폴더 여부 체크하고, 폴더라면 그 안에 chromedriver가 있는지 확인.
    for i in current_list:
        path = os.path.join(os.getcwd(), i) 
        if os.path.isdir(path): 
            if 'chromedriver.exe' in os.listdir(path): 
                chrome_list.append(i) 
    
    # 삭제 대상(구 버전 chromedriver)
    old_version = list(set(chrome_list)-set([chrome_ver])) 
    
    # 구 버전 삭제(폴더 그대로 삭제함.)
    for i in old_version:
        path = os.path.join(os.getcwd(),i) 
        shutil.rmtree(path)
    if not chrome_ver in current_list:
        # 인자를 True로 주면 현재 디렉토리에 최신 버전 install.
        chromedriver_autoinstaller.install(True)

In [ ]:
def switch_to_details():    
    """ 해당 날짜 회의록으로 화면을 전환하는 method. """
    driver.switch_to.window(driver.window_handles[-1])
        
def switch_to_main():
    """ 초기 화면으로 화면을 전환하는 method. """
    driver.switch_to.window(driver.window_handles[0])
        

In [ ]:
class Scrap:
    def __init__(self, url):
        """ columns_name -> 컬럼명 정의. """
        self._url = url
        self._columns_name = ['회의록구분', '대수', '회의구분', '위원회', '회수', '피감사기관', '회의일자', '발언자', '발언순번', '발언내용', '기관구분']
        self._df = pd.DataFrame(columns = self._columns_name)
    
    def open_url(self):
        """ url을 open하는 method. """
        
        driver.implicitly_wait(3)  # 최대 3초까지 기다리는 것을 허용.
        driver.get(self._url)
    
    def proceedings(self):
        """ <서울특별시 회의록> 글자를 return하는 method. """
        
        driver.implicitly_wait(3)
        return driver.find_element(By.XPATH, '//*[@id="header"]/div/h1/a/span/em').text
    
    def conference(self):
        """ <행정사무감사> 를 클릭하고, text를 return하는 method. """
        
        driver.implicitly_wait(3)
        conf = driver.find_element(By.XPATH, '//*[@id="tree"]/ul/li[5]/span/span[3]')
        conf.click()
        return conf.text
    
    def generation(self):
        """제10대 부분을 클릭하고, text에 속해있는 숫자(10)을 return하는 method. 
        10대 뿐 아니라 다른 대까지 반복하고 싶다면 details method 처럼 인자를 받아 반복문을 돌려야 한다. """
        
        ''' iteration by xpath -> //*[@id="tree"]/ul/li[5]/ul/li[%d]/span/span[3] '''
        driver.implicitly_wait(3)
        gen = driver.find_element(By.XPATH, '//*[@id="tree"]/ul/li[5]/ul/li[1]/span/span[3]')    
        gen.click()
        
        # 대수 숫자부분 추출.
        gen_txt = gen.text
        return re.search(r'\d+', gen_txt).group()
                
    def committee(self):
        """ 교육위원회행정사무감사를 클릭하고, 해당하는 위원회를 return 하는 method. """
        
        ''' iteration by xpath -> //*[@id="tree"]/ul/li[5]/ul/li[%d]/ul/li[10]/span/span[3] '''
        
        driver.implicitly_wait(3)
        committee = driver.find_element(By.XPATH, '//*[@id="tree"]/ul/li[5]/ul/li[1]/ul/li[10]/span/span[3]')   
        committee.click() 
        
        return committee.text

    def count(self):
        """ 제 298회 부분을 클릭하고, text를 return하는 method. 
        다른 회도 수집하고 싶다면, details method 처럼 인자를 받아 반복문을 돌려야 한다.         
        
        //*[@id="tree"]/ul/li[5]/ul/li[%d]/ul/li[10]/ul/li[%d]/span/span[3] 방식으로 count, datails를 처리해야 반복 가능!
        
        첫번째 %d : 대에 따른 구분(generation)
        두번째 %d : 회에 따른 구분(count)
        """
        
        driver.implicitly_wait(3)
        cnt = driver.find_element(By.XPATH, '//*[@id="tree"]/ul/li[5]/ul/li[1]/ul/li[10]/ul/li[1]/span/span[3]')  
        cnt.click()
        return cnt.text

    def details(self, i):  
        """ 그 날에 이루어진 구체적인 회의를 클릭하고, 피감사기관, 회의일자를 추출하는 method. """
        '''//*[@id="tree"]/ul/li[5]/ul/li[%d]/ul/li[10]/ul/li[%d]/ul/li[%d]/span/span[3]/a
        첫번째 %d : 대에 따른 구분(generation)
        두번째 %d : 회에 따른 구분(count)
        세번째 %d : 날짜에 따른 구분(date)
        '''
        
        driver.implicitly_wait(3)
        hg = driver.find_element(By.XPATH, '//*[@id="tree"]/ul/li[5]/ul/li[1]/ul/li[10]/ul/li[1]/ul/li[%d]/span/span[3]/a' % i)  
        hg_txt = hg.text[5:]  # [임시] 문구 처리 위한 slicing. 최신 날짜 회의록 이름에 [임시]라는 말이 빠져있으면 슬라이싱 지워주기.
         
        ''' 괄호와 날짜, 요일 형식이 일정하다고 가정하고 [1:-1] 슬라이싱을 통해 각 항목의 괄호 제거.
        strip()을 통해 좌우공백 제거.
        auditee의 경우 글의 시작이 '피감사기관 '이기 때문에 해당부분을 제거하기 위해 [6:] 슬라이싱.

        해당 형식이 바뀌는 경우가 있다면 추출 방식 수정 필요.
        '''
        
        # 피감사기관, 회의일자 추출.
        auditee = hg_txt[:-17][1:-1].strip()[6:]   # 피감사기관
        date = hg_txt[-16:][1:-1].strip()   # 회의일자
        
        hg.click()
        return auditee, date
    
    def scraping(self):
        """ 회의록 내용을 전부 scraping하여 문장들을 모은 list를 return하는 method. """
        
        driver.implicitly_wait(3)
        li = []
        i = 1

        # 더 이상 대화가 없으면 정상적으로 xpath를 가져올 수 없으므로 break가 실행됨.        
        while True:
            try:
                # 발언자 추출. 'ㅇ'을 제외하기 위하여 [1:] 사용.
                person = driver.find_element(By.XPATH, '//*[@id="canvas"]/spk[%d]/strong' % i).text[1:]
        
                # 발언자를 제외한 내용만 추출. [2:] 사용.
                speak = driver.find_element(By.XPATH, '//*[@id="canvas"]/spk[%d]' % i).text.split()[2:]
        
                person_li = person.split()
                
                '''
                위원의 경우 XXX 위원, 
                위원이 아닌 경우 교육감 XXX
                의 규칙으로 작성.
                '''
                try:
                    if person_li[1] == '위원':
                        center = person_li[1]
                    else:
                        center = person_li[0]
                except IndexError:
                    center = None
            
                '''
                {proceedings_txt : 회의록구분, generation_txt : 대수, conference_txt : 회의구분, committee_txt : 위원회
                count_txt : 회수, auditee : 피감사기관, date : 회의일자, person : 발언자, i : 발언순번, speak : 발언내용, center : 기관구분}       
                '''
        
                li.append([proceedings_txt, generation_txt, conference_txt, committee_txt, count_txt, auditee_txt, date_txt, person, i, ' '.join(speak), center])  
            
                i += 1                  
            except NoSuchElementException:                
                print("해당 회의록에 대한 수집을 종료합니다.")
                break
        return li
    
    def make_dataframe(self, li):
        """ 수집한 문장들을 바탕으로 데이터프레임을 만들어 return 하는 method. """
        return pd.DataFrame(scrap_list, columns = self._columns_name)
    
    def concat_df(self, f):
        self._df = pd.concat([self._df, f])      
        
    def clear_df(self):
        self._df = pd.DataFrame(columns = self._columns_name)
    
    def save_file(self, n, p, c):
        """ make_dataframe에서 만든 데이터프레임을 파일로 저장하는 method. """
        filename = p + " " + c + ".csv"
        self._df.to_csv(os.path.join(n, filename), index = False, encoding = 'euc-kr')
        
        

# 서울특별시의회 회의록 사이트 접속

In [ ]:
# selenium 정상 실행을 위한 크롬 드라이버 자동 업데이트
chromedriver_update()
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
location = os.path.join(os.getcwd(), chrome_ver)
driver = webdriver.Chrome(os.path.join(location, 'chromedriver.exe'))

"""
directory_name = "회의록_모음"
이라고 설정하고 스크레이핑한 것 모아두면
단어 빈도 분석, 발언자 별 발언 수 분석 파일과 바로 연관시켜 진행할 수 있습니다. 
"""
directory_name = "회의록_모음"
createDirectory(directory_name)

u = 'https://ms.smc.seoul.kr/kr/assembly/committee.do'
s = Scrap(u)

s.open_url()

# 회의록 구분 (서울특별시의회 회의록) 텍스트 추출
proceedings_txt = s.proceedings()

# 행정사무감사 클릭       
conference_txt = s.conference()

# 제 N대 회의... 클릭
generation_txt = s.generation()
        
# 교육위원회행정사무감사 클릭          
committee_txt = s.committee()    
        
# 제 N회 정례회 클릭.
count_txt = s.count()

idx = 1
while True:
    try:
        # 피감사기관 ~ 으로 시작하는 구체적인 회의록 클릭.        
        auditee_txt, date_txt = s.details(idx)
        idx += 1
        
        switch_to_details()
        
        # 팝업창 무시하기
        da = Alert(driver)
        da.accept()
        
        scrap_list = s.scraping()
        
        df = s.make_dataframe(scrap_list)
        
        s.concat_df(df)
        
        print("%s 저장이 완료되었습니다." % (proceedings_txt + count_txt + date_txt))
        
        driver.close()
        
        switch_to_main()
        
    except NoSuchElementException:
        print("스크레이핑을 종료합니다.")
        break

        
s.save_file(directory_name, proceedings_txt, count_txt)

